###### Because of the memory constraint of 10 Mb and because the full dataset weight 10.3 Mb, we can't load all in pandas data frame since pandas requires 4 to 6 times amount of space in memory as the file does on disk (especially when there's many string columns). 

###### We gonna a need to load in chunks, 5000 (10 times less than the full data set which have 50000 rows):

In [1]:
import pandas as pd
import numpy as np
import sqlite3
%matplotlib inline

# Insights:
chunk_iter_3 = pd.read_csv("crunchbase-investments.csv",\
                         chunksize = 3,\
                        encoding='ISO-8859-1')
for chunk in chunk_iter_3:
    df = chunk
    break
df

,company_permalink,company_name,company_category_code,company_country_code,company_state_code,company_region,company_city,investor_permalink,investor_name,investor_category_code,investor_country_code,investor_state_code,investor_region,investor_city,funding_round_type,funded_at,funded_month,funded_quarter,funded_year,raised_amount_usd
0,/company/advercar,AdverCar,advertising,USA,CA,SF Bay,San Francisco,/company/1-800-flowers-com,1-800-FLOWERS.COM,NaN,USA,NY,New York,New York,series-a,2012-10-30,2012-10,2012-Q4,2012,2000000
1,/company/launchgram,LaunchGram,news,USA,CA,SF Bay,Mountain View,/company/10xelerator,10Xelerator,finance,USA,OH,Columbus,Columbus,other,2012-01-23,2012-01,2012-Q1,2012,20000
2,/company/utap,uTaP,messaging,USA,NaN,United States - Other,NaN,/company/10xelerator,10Xelerator,finance,USA,OH,Columbus,Columbus,other,2012-01-01,2012-01,2012-Q1,2012,20000


###### Columns with redundant data: "funded month", "funded year", "funded quarter" as all the info about date foundation is in "funded_at".

In [2]:
info_cols = {}
col_types = {}

for chunk in chunk_iter_3:
    columns = chunk.columns
    for col in columns:
        if col not in info_cols:
            info_cols[col] = [0,0]
            col_types[col] = str(chunk[col].dtype)
    break

chunk_iter = pd.read_csv("crunchbase-investments.csv",\
                         chunksize = 5000,\
                        encoding='ISO-8859-1')

total_memory = 0
total_rows = 0

for chunk in chunk_iter:
    for col in columns:
        info_cols[col][0] += chunk[col].isnull().sum()
        memory = chunk[col].memory_usage(deep=True)
        info_cols[col][1] += memory
        total_memory += memory
    total_rows += chunk.shape[0]
        
print("The total rows of the data set is: ", total_rows)
print("Total memory usage for the data set: ",np.round(total_memory/(1024*1024),2)," Megabytes")

The total rows of the data set is:  52870
Total memory usage for the data set:  57.01  Megabytes


In [3]:
info_cols
# format of the dict 'column name':[null-values,bytes]

{'company_category_code': [643, 3422024],
 'company_city': [533, 3506846],
 'company_country_code': [1, 3173096],
 'company_name': [1, 3592246],
 'company_permalink': [1, 4058708],
 'company_region': [1, 3412505],
 'company_state_code': [492, 3106971],
 'funded_at': [3, 3543105],
 'funded_month': [3, 3384504],
 'funded_quarter': [3, 3384504],
 'funded_year': [3, 423880],
 'funding_round_type': [3, 3411627],
 'investor_category_code': [50427, 623344],
 'investor_city': [12480, 2886003],
 'investor_country_code': [12001, 2648212],
 'investor_name': [2, 3916586],
 'investor_permalink': [2, 4981468],
 'investor_region': [2, 3397201],
 'investor_state_code': [16809, 2477527],
 'raised_amount_usd': [3599, 423880]}

In [4]:
col_types
# Format of the dict 'column name':type

{'company_category_code': 'object',
 'company_city': 'object',
 'company_country_code': 'object',
 'company_name': 'object',
 'company_permalink': 'object',
 'company_region': 'object',
 'company_state_code': 'object',
 'funded_at': 'object',
 'funded_month': 'object',
 'funded_quarter': 'object',
 'funded_year': 'int64',
 'funding_round_type': 'object',
 'investor_category_code': 'object',
 'investor_city': 'object',
 'investor_country_code': 'object',
 'investor_name': 'object',
 'investor_permalink': 'object',
 'investor_region': 'object',
 'investor_state_code': 'object',
 'raised_amount_usd': 'int64'}

In [6]:
# Unique values for object columns.
obj_cols = []
unique_values_object_column = {}

chunk_iter = pd.read_csv("crunchbase-investments.csv",\
                         chunksize = 5000,\
                        encoding='ISO-8859-1')

for k,v in col_types.items():
    if v == 'object':
        obj_cols.append(k)
        
for col in obj_cols: 
    if col not in unique_values_object_column:
        unique_values_object_column[col] = set()

for chunk in chunk_iter:
    for col in obj_cols:
        unique_values = chunk[col].unique()
        for value in unique_values:
            unique_values_object_column[col].add(value)

for k,v in unique_values_object_column.items():
    unique_values_object_column[k] = len(v)
    
unique_values_object_column    
# Format of the dict 'column object name': unique-values

{'company_category_code': 44,
 'company_city': 1230,
 'company_country_code': 3,
 'company_name': 11574,
 'company_permalink': 11574,
 'company_region': 547,
 'company_state_code': 51,
 'funded_at': 2809,
 'funded_month': 193,
 'funded_quarter': 73,
 'funding_round_type': 10,
 'investor_category_code': 44,
 'investor_city': 993,
 'investor_country_code': 75,
 'investor_name': 10466,
 'investor_permalink': 10553,
 'investor_region': 586,
 'investor_state_code': 53}

###### First conclusions: 

- Total memory usage of the data set: 57 Mb
- Total rows of the data set: 52870
- The numeric columns marked as int64 actually are float because every column has null values when we compute the full data-set. 
- Every object column have less than 50% of unique values. 


In [7]:
float_cols = []

for k,v in col_types.items():
    if v == 'int64':
        float_cols.append(k)
        
float_cols

['funded_year', 'raised_amount_usd']

##### object columns to categoric columns: we can convert these columns into categories, but according to the type of information provided by the columns the following columns shouldn't be categorized:

- 'company_name', 'company_permalink', 'funded_at', 'investor_name', 'investor_permalink'. 

In [8]:
drop_columns = ["funded_month", "funded_year", "funded_quarter"]
not_categoric = ['company_name', 'company_permalink', 'funded_at', 'investor_name', 'investor_permalink']
to_categoric = ['funding_round_type','company_region','investor_state_code',
 'investor_category_code',
 'company_country_code',
 'company_city','company_state_code',
 'investor_country_code','company_category_code','investor_region',
 'investor_city']


In [9]:
# optimizing data types

chunk_iter = pd.read_csv("crunchbase-investments.csv",\
                         chunksize = 5000,\
                        encoding='ISO-8859-1')

total_memory = 0

for chunk in chunk_iter:
    for col in float_cols:
        chunk[col] = pd.to_numeric(chunk[col])
        memory = chunk[col].memory_usage(deep=True)
        total_memory += memory
        
    for col in to_categoric: 
        chunk[col] = chunk[col].astype('category')
        memory = chunk[col].memory_usage(deep=True)
        total_memory += memory
    chunk["funded_at"] = pd.to_datetime(chunk["funded_at"])  
    chunk.drop(columns=drop_columns, inplace=True)
    memory = chunk.memory_usage(deep=True).sum()
    total_memory += memory 
        
print("Total memory usage for the data set: ",np.round(total_memory/(1024*1024),2)," Megabytes")

Total memory usage for the data set:  21.34  Megabytes


In [10]:
# Create and connect to a new SQLite database file.

conn = sqlite3.connect('crunchbase.db')

# optimizing data types


chunk_iter = pd.read_csv("crunchbase-investments.csv",\
                         chunksize = 5000,\
                        encoding='ISO-8859-1')

for chunk in chunk_iter:
    for col in float_cols:
        chunk[col] = pd.to_numeric(chunk[col])
        memory = chunk[col].memory_usage(deep=True)
        total_memory += memory
        
    for col in to_categoric: 
        chunk[col] = chunk[col].astype('category')
        
    chunk["funded_at"] = pd.to_datetime(chunk["funded_at"])  
    chunk.drop(columns=drop_columns, inplace=True)
    
    chunk.to_sql('exhibitions',conn,if_exists='append',index=False) 


In [11]:
results_df = pd.read_sql('PRAGMA table_info(exhibitions);',conn)
results_df

,cid,name,type,notnull,dflt_value,pk
0,0,company_permalink,TEXT,0,None,0
1,1,company_name,TEXT,0,None,0
2,2,company_category_code,TEXT,0,None,0
3,3,company_country_code,TEXT,0,None,0
4,4,company_state_code,TEXT,0,None,0
5,5,company_region,TEXT,0,None,0
6,6,company_city,TEXT,0,None,0
7,7,investor_permalink,TEXT,0,None,0
8,8,investor_name,TEXT,0,None,0
9,9,investor_category_code,TEXT,0,None,0


In [12]:
!wc crunchbase.db

   10318   572854 20422656 crunchbase.db


a) What proportion of the total amount of funds did the top 10% raise? 
b) What about the top 1%? 
c) Compare these values to the proportions the bottom 10% and bottom 1% raised.

In [32]:
query1 = 'SELECT SUM(raised_amount_usd) FROM exhibitions'

query2 = 'SELECT company_name, COUNT(company_name) FROM exhibitions GROUP BY company_name'

query3 = 'SELECT CAST(SUM(raised_amount_usd) as Float) total_raise, company_name FROM exhibitions \
GROUP BY company_name ORDER BY total_raise DESC LIMIT {0};'

query4 = 'SELECT CAST(SUM(raised_amount_usd) as Float) total_raise, company_name FROM exhibitions \
GROUP BY company_name ORDER BY total_raise ASC LIMIT {0};'


total_amount = pd.read_sql(query1,conn)
total_companies = pd.read_sql(query2,conn).shape[0]
ten_percent_companies = int(total_companies*0.1)

preg_a_top = pd.read_sql(query3.format(ten_percent_companies),conn)
preg_a_bottom = pd.read_sql(query4.format(ten_percent_companies),conn)

In [38]:
# Answer: 

print("proportion of the total amount of funds did the top 10% raise?")
print(preg_a_top["total_raise"].sum()*100/total_amount)
print(" ")
print("What about the top 1%?")
print(preg_a_top.loc[:110,"total_raise"].sum()*100/total_amount)


proportion of the total amount of funds did the top 10% raise?
   SUM(raised_amount_usd)
0               67.127727
 
What about the top 1%?
   SUM(raised_amount_usd)
0               25.835418


In [37]:
preg_a_top.loc[:110,"total_raise"].sum()*100/total_amount

,SUM(raised_amount_usd)
0,25.835418


##### Which category of company attracted the most investments?

In [40]:
query5 = 'SELECT CAST(SUM(raised_amount_usd) as Float) total_raise, company_category_code FROM exhibitions \
GROUP BY company_category_code ORDER BY total_raise DESC LIMIT 10;'

preg_b = pd.read_sql(query5,conn)
preg_b

,total_raise,company_category_code
0,2.207928e+11,biotech
1,1.461690e+11,software
2,1.295548e+11,mobile
3,1.054105e+11,cleantech
4,9.172185e+10,enterprise
5,8.028653e+10,web
6,5.073421e+10,medical
7,5.015332e+10,advertising
8,4.513444e+10,ecommerce
9,4.483937e+10,network_hosting


##### Which investor contributed the most money (across all startups)?

In [41]:
query6 = 'SELECT CAST(SUM(raised_amount_usd) as Float) total_raise, investor_name FROM exhibitions \
GROUP BY investor_name ORDER BY total_raise DESC LIMIT 10;'

preg_c = pd.read_sql(query6,conn)
preg_c

,total_raise,investor_name
0,2.243565e+10,Kleiner Perkins Caufield & Byers
1,1.938508e+10,New Enterprise Associates
2,1.294425e+10,Accel Partners
3,1.275092e+10,Goldman Sachs
4,1.207880e+10,Sequoia Capital
5,1.193840e+10,Intel
6,1.161760e+10,Google
7,1.146000e+10,Time Warner
8,1.133800e+10,Comcast
9,9.921966e+09,Greylock Partners


##### Which investors contributed the most money per startup?

In [42]:
query7 = 'SELECT CAST(SUM(raised_amount_usd) as Float) total_raise, investor_name, company_name FROM exhibitions \
GROUP BY company_name ORDER BY total_raise DESC LIMIT 10;'

preg_d = pd.read_sql(query7,conn)
preg_d

,total_raise,investor_name,company_name
0,5.936000e+10,Eagle River Holdings,Clearwire
1,2.037080e+10,Ted Leonsis,Groupon
2,9.010000e+09,Christian Reitberger,Nanosolar
3,8.308200e+09,Reid Hoffman,Facebook
4,6.500000e+09,Dave Goldberg,SurveyMonkey
5,5.772026e+09,Reid Hoffman,Zynga
6,5.576000e+09,U.S. Department of Energy,Fisker Automotive
7,5.528800e+09,Pejman Nozad,Dropbox
8,5.370000e+09,Steve Case,LivingSocial
9,5.200000e+09,Marlin Equity Partners,sigmacare


funding_round_type
##### Which funding round was the most popular? Which was the least popular?

In [44]:
query8 = 'SELECT COUNT(investor_name) popularity, funding_round_type FROM exhibitions \
GROUP BY funding_round_type ORDER BY popularity DESC LIMIT 10;'

preg_e = pd.read_sql(query8,conn)
preg_e

,popularity,funding_round_type
0,27876,series-a
1,21740,series-c+
2,17978,angel
3,17834,venture
4,17588,series-b
5,1928,other
6,714,private-equity
7,66,post-ipo
8,10,crowdfunding
9,2,None
